In [ ]:
import os
import pandas as pd
import numpy as np
import urllib.request

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## Pre Processing original dataset

In [ ]:
dfs = pd.read_csv("drive/MyDrive/Wikipedia-Image-Captioning/wit_train_farsi.csv")
dfs

,language,page_url,image_url,page_title,section_title,hierarchical_section_title,caption_reference_description,caption_attribution_description,caption_alt_text_description,mime_type,original_height,original_width,is_main_image,attribution_passes_lang_id,page_changed_recently,context_page_description,context_section_description
0,fa,https://fa.wikipedia.org/wiki/%D8%B6%D8%B1%D8%...,https://upload.wikimedia.org/wikipedia/commons...,ضرابخانه سلطنتی,تاریخچه,ضرابخانه سلطنتی / تاریخچه,ساختمان ضرابخانه سلطنتی از ۱۸۸۰,English: The old Royal Mint building Royal Min...,NaN,image/jpeg,480,640,False,False,False,ضرابخانه سلطنتی یک نهاد مجاز برای ضرب سکه در ب...,NaN
1,fa,https://fa.wikipedia.org/wiki/%D9%81%D9%87%D8%...,https://upload.wikimedia.org/wikipedia/commons...,فهرست آثار ملی شهرستان تهران,NaN,فهرست آثار ملی شهرستان تهران,تصاویر بیشتر,فارسی: باغ و عمارت عین‌الدوله English: Einoldo...,NaN,image/jpeg,720,1280,False,True,False,این فهرستی از آثار ثبتی شهرستان تهران است.,این فهرستی از آثار ثبتی شهرستان تهران است.
2,fa,https://fa.wikipedia.org/wiki/%DA%A9%D9%84%DB%...,http://upload.wikimedia.org/wikipedia/commons/...,کلیسای کاشوتی,NaN,کلیسای کاشوتی,NaN,Kashveti Church in Tbilisi,NaN,image/jpeg,616,741,True,False,False,کلیسای کاشوتی جرجیس یک کلیسای ارتدکس گرجی است ...,کلیسای کاشوتی جرجیس (گرجی: ქაშვეთის წმინდა გიო...
3,fa,https://fa.wikipedia.org/wiki/%D8%B1%D9%87%D8%...,https://upload.wikimedia.org/wikipedia/commons...,رهبران زن,Elected female heads of state or government,رهبران زن / Elected female heads of state or g...,NaN,For documentary purposes the German Federal Ar...,NaN,image/jpeg,792,552,False,False,True,رهبران زن به زنانی اطلاق می‌شود که در تاریخ کش...,{{زیرنویس نقشه|#FFC0CB|متصدی head of state or ...
4,fa,https://fa.wikipedia.org/wiki/%DA%A9%D8%A7%D8%...,http://upload.wikimedia.org/wikipedia/commons/...,کارلوس مویا,NaN,کارلوس مویا,NaN,Thomas 'Indianas' Bohane at his quarterfinal m...,NaN,image/jpeg,2136,1370,True,False,False,کارلوس مویا یک تنیس‌باز اهل اسپانیا است.,کارلوس مویا (کاتالونیایی: Carles Moyà Llompart...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485538,fa,https://fa.wikipedia.org/wiki/%D8%B2%D9%86%D8%...,http://upload.wikimedia.org/wikipedia/commons/...,زنجیره مارکوف,NaN,زنجیره مارکوف,نمونه‌ای از زنجیره مارکوف با سه حالت,NaN,NaN,image/svg+xml,170,175,False,False,False,زنجیره مارکوف مدلی تصادفی برای توصیف یک توالی ...,زنجیره مارکوف مدلی تصادفی برای توصیف یک توالی ...
485539,fa,https://fa.wikipedia.org/wiki/%DA%A9%D9%88%D8%...,https://upload.wikimedia.org/wikipedia/commons...,کوررنگی,علت,کوررنگی / علت,منحنی نشان‌دهندهٔ میزان تحریک هر کدام از ۳ نوع...,The Stockman and Sharpe (2000) 2° cone fundame...,NaN,image/png,501,772,False,False,False,کوررنگی یک بیماری اختلال ارثی است که در آن فرد...,سلول‌های مخروطی نوعی از سلول‌های گیرندهٔ نور ه...
485540,fa,https://fa.wikipedia.org/wiki/%D9%85%D9%88%D8%...,https://upload.wikimedia.org/wikipedia/commons...,موش صحرایی ژاپنی بزرگ,NaN,موش صحرایی ژاپنی بزرگ,NaN,Apodemus speciosus(Large Japanese field mouse),NaN,image/jpeg,450,600,True,False,False,موش صحرایی ژاپنی بزرگ نام یک گونه از تیره موشا...,موش صحرایی ژاپنی بزرگ (نام علمی: Apodemus spec...
485541,fa,https://fa.wikipedia.org/wiki/%DA%AF%D8%A7%D8%...,https://upload.wikimedia.org/wikipedia/commons...,گادنی,NaN,گادنی,NaN,Pill Box in Upper Godney,NaN,image/jpeg,640,480,True,False,False,گادنی یک روستا و محله مدنی در بریتانیا است که ...,گادنی (به انگلیسی: Godney) یک روستا و محله مدن...


In [ ]:
dfs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 485543 entries, 0 to 485542
Data columns (total 17 columns):
 #   Column                           Non-Null Count   Dtype 
---  ------                           --------------   ----- 
 0   language                         485543 non-null  object
 1   page_url                         485543 non-null  object
 2   image_url                        485543 non-null  object
 3   page_title                       485543 non-null  object
 4   section_title                    179301 non-null  object
 5   hierarchical_section_title       485543 non-null  object
 6   caption_reference_description    148095 non-null  object
 7   caption_attribution_description  466724 non-null  object
 8   caption_alt_text_description     73151 non-null   object
 9   mime_type                        485543 non-null  object
 10  original_height                  485543 non-null  int64 
 11  original_width                   485543 non-null  int64 
 12  is_main_image   

In [ ]:
pd.options.display.max_colwidth = 50

In [ ]:
dfs.iloc[3, 1]

'https://fa.wikipedia.org/wiki/%D8%B1%D9%87%D8%A8%D8%B1%D8%A7%D9%86_%D8%B2%D9%86'

In [ ]:
dfs.iloc[3, 2]

'https://upload.wikimedia.org/wikipedia/commons/8/8f/Bundesarchiv_Bild_183-1987-0305-114%2C_Berlin%2C_XII._DFD-Kongress.jpg'

In [ ]:
dfs2 = pd.DataFrame.copy(dfs.loc[dfs.caption_reference_description.isna() == False])

In [ ]:
dfs2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 148095 entries, 0 to 485539
Data columns (total 17 columns):
 #   Column                           Non-Null Count   Dtype 
---  ------                           --------------   ----- 
 0   language                         148095 non-null  object
 1   page_url                         148095 non-null  object
 2   image_url                        148095 non-null  object
 3   page_title                       148095 non-null  object
 4   section_title                    50914 non-null   object
 5   hierarchical_section_title       148095 non-null  object
 6   caption_reference_description    148095 non-null  object
 7   caption_attribution_description  129276 non-null  object
 8   caption_alt_text_description     10459 non-null   object
 9   mime_type                        148095 non-null  object
 10  original_height                  148095 non-null  int64 
 11  original_width                   148095 non-null  int64 
 12  is_main_image   

In [ ]:
dfs.columns[np.sum(dfs2.isna()) > 0]

Index(['section_title', 'caption_attribution_description',
       'caption_alt_text_description', 'context_page_description',
       'context_section_description'],
      dtype='object')

In [ ]:
dropable = ['language', 'section_title', 'hierarchical_section_title', 'caption_alt_text_description', 'mime_type', 'original_height', 'original_width',
            'is_main_image', 'attribution_passes_lang_id', 'page_changed_recently', 'context_section_description', 'caption_attribution_description']

In [ ]:
dfs2.drop(dropable, axis=1, inplace=True)

In [ ]:
dfs2

,page_url,image_url,page_title,caption_reference_description,context_page_description
0,https://fa.wikipedia.org/wiki/%D8%B6%D8%B1%D8%...,https://upload.wikimedia.org/wikipedia/commons...,ضرابخانه سلطنتی,ساختمان ضرابخانه سلطنتی از ۱۸۸۰,ضرابخانه سلطنتی یک نهاد مجاز برای ضرب سکه در ب...
1,https://fa.wikipedia.org/wiki/%D9%81%D9%87%D8%...,https://upload.wikimedia.org/wikipedia/commons...,فهرست آثار ملی شهرستان تهران,تصاویر بیشتر,این فهرستی از آثار ثبتی شهرستان تهران است.
8,https://fa.wikipedia.org/wiki/%D9%84%D9%88%D8%...,https://upload.wikimedia.org/wikipedia/commons...,لوتار فون مایر,یولیوس لوتار مایر,یولیوس لوتار فون مایر شیمی‌دان آلمانی هم دورهٔ...
10,https://fa.wikipedia.org/wiki/%D9%81%DB%8C%D9%...,https://upload.wikimedia.org/wikipedia/commons...,فینال مسابقات باشگاهی قهرمانی آسیا ۱۹۷۰,تماشاگران از ساعت‌ها پیش از آغاز بازی در ورزشگ...,فینال مسابقات باشگاهی قهرمانی آسیا ۱۹۷۰ یک مسا...
15,https://fa.wikipedia.org/wiki/%D8%B4%DB%8C%D8%...,https://upload.wikimedia.org/wikipedia/commons...,شیء ناشناس پرنده,عکس گرفته‌شده از تصویر احتمالیِ یک بشقاب‌پرنده...,شیءِ ناشناسِ پرنده، که در فارسی به آن بشقاب پر...
...,...,...,...,...,...
485529,https://fa.wikipedia.org/wiki/%D9%84%DB%8C%D9%...,https://upload.wikimedia.org/wikipedia/commons...,لیناردو نایتو دی ماکیدو,Netto being presented by Hunan Billows,لیناردو نایتو دی ماکیدو مهاجم اهل برزیل است که...
485534,https://fa.wikipedia.org/wiki/%D9%BE%D8%B1%DA%...,https://upload.wikimedia.org/wikipedia/commons...,پرچم لهستان,گونه دیگر از پرچم لهستان,پرچم لهستان از دو نوار افقی هم عرض تشکیل شده‌ا...
485537,https://fa.wikipedia.org/wiki/%D8%B5%D8%A7%D9%...,https://upload.wikimedia.org/wikipedia/commons...,صالح کوتاه,نقشه دیتر امان فرانسوی از اندیکا در سال 1910 (...,صالح کوتاه، روستایی از توابع شهرستان اندیکا در...
485538,https://fa.wikipedia.org/wiki/%D8%B2%D9%86%D8%...,http://upload.wikimedia.org/wikipedia/commons/...,زنجیره مارکوف,نمونه‌ای از زنجیره مارکوف با سه حالت,زنجیره مارکوف مدلی تصادفی برای توصیف یک توالی ...


In [ ]:
dfs2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 148095 entries, 0 to 485539
Data columns (total 5 columns):
 #   Column                         Non-Null Count   Dtype 
---  ------                         --------------   ----- 
 0   page_url                       148095 non-null  object
 1   image_url                      148095 non-null  object
 2   page_title                     148095 non-null  object
 3   caption_reference_description  148095 non-null  object
 4   context_page_description       148070 non-null  object
dtypes: object(5)
memory usage: 6.8+ MB


In [ ]:
dfs2.loc[dfs2.context_page_description.isna()]

,page_url,image_url,page_title,caption_reference_description,context_page_description


In [ ]:
dfs2.dropna(inplace=True)

In [ ]:
dfs2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 148070 entries, 0 to 485539
Data columns (total 5 columns):
 #   Column                         Non-Null Count   Dtype 
---  ------                         --------------   ----- 
 0   page_url                       148070 non-null  object
 1   image_url                      148070 non-null  object
 2   page_title                     148070 non-null  object
 3   caption_reference_description  148070 non-null  object
 4   context_page_description       148070 non-null  object
dtypes: object(5)
memory usage: 6.8+ MB


In [ ]:
dfs2.loc[3921].image_url

'http://upload.wikimedia.org/wikipedia/commons/5/5a/Santa_Fe_NM.jpg'

In [ ]:
dfs2.to_csv("./wiki_ic.csv", index=False)

In [ ]:
drive.flush_and_unmount()

In [ ]:
!mv wiki_ic.csv drive/MyDrive/Wikipedia-Image-Captioning/

## Cleaning data 🧹

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/wikiPedia_ic/wiki_ic.csv")
df

,page_url,image_url,page_title,caption_reference_description,context_page_description
0,https://fa.wikipedia.org/wiki/%D8%B6%D8%B1%D8%...,https://upload.wikimedia.org/wikipedia/commons...,ضرابخانه سلطنتی,ساختمان ضرابخانه سلطنتی از ۱۸۸۰,ضرابخانه سلطنتی یک نهاد مجاز برای ضرب سکه در ب...
1,https://fa.wikipedia.org/wiki/%D9%81%D9%87%D8%...,https://upload.wikimedia.org/wikipedia/commons...,فهرست آثار ملی شهرستان تهران,تصاویر بیشتر,این فهرستی از آثار ثبتی شهرستان تهران است.
2,https://fa.wikipedia.org/wiki/%D9%84%D9%88%D8%...,https://upload.wikimedia.org/wikipedia/commons...,لوتار فون مایر,یولیوس لوتار مایر,یولیوس لوتار فون مایر شیمی‌دان آلمانی هم دورهٔ...
3,https://fa.wikipedia.org/wiki/%D9%81%DB%8C%D9%...,https://upload.wikimedia.org/wikipedia/commons...,فینال مسابقات باشگاهی قهرمانی آسیا ۱۹۷۰,تماشاگران از ساعت‌ها پیش از آغاز بازی در ورزشگ...,فینال مسابقات باشگاهی قهرمانی آسیا ۱۹۷۰ یک مسا...
4,https://fa.wikipedia.org/wiki/%D8%B4%DB%8C%D8%...,https://upload.wikimedia.org/wikipedia/commons...,شیء ناشناس پرنده,عکس گرفته‌شده از تصویر احتمالیِ یک بشقاب‌پرنده...,شیءِ ناشناسِ پرنده، که در فارسی به آن بشقاب پر...
...,...,...,...,...,...
148065,https://fa.wikipedia.org/wiki/%D9%84%DB%8C%D9%...,https://upload.wikimedia.org/wikipedia/commons...,لیناردو نایتو دی ماکیدو,Netto being presented by Hunan Billows,لیناردو نایتو دی ماکیدو مهاجم اهل برزیل است که...
148066,https://fa.wikipedia.org/wiki/%D9%BE%D8%B1%DA%...,https://upload.wikimedia.org/wikipedia/commons...,پرچم لهستان,گونه دیگر از پرچم لهستان,پرچم لهستان از دو نوار افقی هم عرض تشکیل شده‌ا...
148067,https://fa.wikipedia.org/wiki/%D8%B5%D8%A7%D9%...,https://upload.wikimedia.org/wikipedia/commons...,صالح کوتاه,نقشه دیتر امان فرانسوی از اندیکا در سال 1910 (...,صالح کوتاه، روستایی از توابع شهرستان اندیکا در...
148068,https://fa.wikipedia.org/wiki/%D8%B2%D9%86%D8%...,http://upload.wikimedia.org/wikipedia/commons/...,زنجیره مارکوف,نمونه‌ای از زنجیره مارکوف با سه حالت,زنجیره مارکوف مدلی تصادفی برای توصیف یک توالی ...


In [ ]:
!pip -q install hazm
!pip -q install clean-text[gpl]


from hazm import Normalizer
from hazm import WordTokenizer
from cleantext import clean
import re

tokenizer = WordTokenizer()
normalizer = Normalizer()

def cleaning(x):
    # clean html, css, js
    x = re.sub(re.compile('<.*?>'), '', x)
    x = re.compile(
        '<\s*style[^>]*>.*?<\s*/\s*style\s*>', re.S | re.I).sub('', x)
    x = re.compile(
        '<\s*script[^>]*>.*?<\s*/\s*script\s*>', re.S | re.I).sub('', x)
    # # regular cleaning
    x = clean(x,
              fix_unicode=True,
              to_ascii=False,
              lower=True,
              no_line_breaks=True,
              no_urls=True,
              no_emails=True,
              no_phone_numbers=True,
              no_numbers=False,
              no_digits=False,
              no_currency_symbols=True,
              no_punct=False,
              replace_with_url=" ",
              replace_with_email=" ",
              replace_with_phone_number=" ",
              replace_with_number=" ",
              replace_with_digit="",
              replace_with_currency_symbol=" ",
              )

    x = normalizer.normalize(x)
    # removing wierd patterns
    wierd_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u'\U00010000-\U0010ffff'
                               u"\u200d"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\u3030"
                               u"\ufe0f"
                               u"\u2069"
                               u"\u2066"
                               u"\u2068"
                               u"\u2067"
                               # u"\u200c"  # half space
                               "]+", flags=re.UNICODE)

    x = wierd_pattern.sub(r'', x)
    # remove #
    x = re.sub("#", " ", x)
    # remove @
    x = re.sub("@", " ", x)
    # remove extra space
    x = re.sub("\s+", " ", x)
    return x.strip()

def text_preprocessor(texts):
  cleaned = []
  for t in texts:
    tokens = tokenizer.tokenize(cleaning(normalizer.normalize(t)))
    cleaned.append(' '.join(tokens))
  return cleaned

     |████████████████████████████████| 316 kB 9.6 MB/s 
     |████████████████████████████████| 233 kB 56.7 MB/s 
     |████████████████████████████████| 1.4 MB 46.6 MB/s 
     |████████████████████████████████| 170 kB 9.9 MB/s 
     |████████████████████████████████| 64 kB 3.1 MB/s 
     |████████████████████████████████| 235 kB 59.9 MB/s 


In [ ]:
cleaned_captions = text_preprocessor(df.caption_reference_description.values)

In [ ]:
cleaned_titles = text_preprocessor(df.page_title.values)

In [ ]:
len(cleaned_captions), len(cleaned_titles)

(148070, 148070)

In [ ]:
np.unique(df.caption_reference_description == cleaned_captions)

array([ True])

In [ ]:
df.page_title = cleaned_titles
np.unique(df.page_title == cleaned_titles)

array([ True])

In [ ]:
include_en = df.loc[df['caption_reference_description'].str.contains('[A-Za-z]', regex=True)].index
df.loc[include_en, 'caption_reference_description'] = df.loc[include_en, 'page_title']

In [ ]:
df.drop(df.loc[df['caption_reference_description'].str.contains('[A-Za-z]', regex=True)].index, axis=0, inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 147887 entries, 0 to 148069
Data columns (total 5 columns):
 #   Column                         Non-Null Count   Dtype 
---  ------                         --------------   ----- 
 0   page_url                       147887 non-null  object
 1   image_url                      147887 non-null  object
 2   page_title                     147887 non-null  object
 3   caption_reference_description  147887 non-null  object
 4   context_page_description       147887 non-null  object
dtypes: object(5)
memory usage: 6.8+ MB


In [ ]:
df.loc[df['caption_reference_description'].str.contains('[A-Za-z]', regex=True)]

,page_url,image_url,page_title,caption_reference_description,context_page_description


In [ ]:
df.drop(['page_url', 'context_page_description'], axis=1, inplace=True)

In [ ]:
df.to_csv('/content/drive/MyDrive/wikiPedia_ic/wiki_fa_cap_cleaned.csv', index=False)

#### Fix data formats

* .svg, .gifs dropped ✔
* Convert to jpg func. defined ✔
* Download all images with `lowercase` postfix characters ✔

In [ ]:
cleaned_df = pd.read_csv("/content/drive/MyDrive/wikiPedia_ic/wiki_fa_cap_cleaned.csv")

In [ ]:
cleaned_df

,image_url,page_title,caption_reference_description
0,https://upload.wikimedia.org/wikipedia/commons...,ضرابخانه سلطنتی,ساختمان ضرابخانه سلطنتی از ۱۸۸۰
1,https://upload.wikimedia.org/wikipedia/commons...,فهرست آثار ملی شهرستان تهران,تصاویر بیشتر
2,https://upload.wikimedia.org/wikipedia/commons...,لوتار فون مایر,یولیوس لوتار مایر
3,https://upload.wikimedia.org/wikipedia/commons...,فینال مسابقات باشگاهی قهرمانی آسیا ۱۹۷۰,تماشاگران از ساعت‌ها پیش از آغاز بازی در ورزشگ...
4,https://upload.wikimedia.org/wikipedia/commons...,شیء ناشناس پرنده,عکس گرفته‌شده از تصویر احتمالی یک بشقاب‌پرنده ...
...,...,...,...
147882,https://upload.wikimedia.org/wikipedia/commons...,لیناردو نایتو دی ماکیدو,لیناردو نایتو دی ماکیدو
147883,https://upload.wikimedia.org/wikipedia/commons...,پرچم لهستان,گونه دیگر از پرچم لهستان
147884,https://upload.wikimedia.org/wikipedia/commons...,صالح کوتاه,نقشه دیتر امان فرانسوی از اندیکا در سال ۱۹۱۰ (...
147885,http://upload.wikimedia.org/wikipedia/commons/...,زنجیره مارکوف,نمونه‌ای از زنجیره مارکوف با سه حالت


In [ ]:
postfixes = []
for i in cleaned_df['image_url'].str.split('.'):
  postfixes.append(i[-1])

In [ ]:
postfixes.count('webp')

8

In [ ]:
postfixes.count('SVG')

12

In [ ]:
postfixes.count('Gif')

1

In [ ]:
np.unique(postfixes)

array(['GIF', 'Gif', 'JPEG', 'JPG', 'Jpeg', 'Jpg', 'PNG', 'SVG', 'gif',
       'jpeg', 'jpg', 'png', 'svg', 'webp'], dtype='<U4')

In [ ]:
cleaned_df[cleaned_df['image_url'].str.endswith('.webp')].index

Int64Index([6631, 21081, 25785, 35865, 95851, 99360, 109005, 144367], dtype='int64')

In [ ]:
# Drop Gifs with .gif, .GIF, .Gif formats
cleaned_df.drop(cleaned_df[cleaned_df['image_url'].str.endswith('.Gif')].index, axis=0, inplace=True)

In [ ]:
# Drop SVGs formats .svg, .SVG
cleaned_df.drop(cleaned_df[cleaned_df['image_url'].str.endswith('.SVG')].index, axis=0, inplace=True)

In [ ]:
cleaned_df.shape

(137170, 3)

In [ ]:
p2 = []
for i in cleaned_df['image_url'].str.split('.'):
  p2.append(i[-1])

In [ ]:
np.unique(p2)

array(['JPEG', 'JPG', 'Jpeg', 'Jpg', 'PNG', 'jpeg', 'jpg', 'png', 'webp'],
      dtype='<U4')

In [ ]:
cleaned_df.to_csv("/content/drive/MyDrive/wikiPedia_ic/wiki_fa_cap_cleaned.csv", index=False)

#### Convert to jpg

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/wikiPedia_ic/wiki_fa_cap_cleaned.csv")
df

,image_url,page_title,caption_reference_description
0,https://upload.wikimedia.org/wikipedia/commons...,ضرابخانه سلطنتی,ساختمان ضرابخانه سلطنتی از ۱۸۸۰
1,https://upload.wikimedia.org/wikipedia/commons...,فهرست آثار ملی شهرستان تهران,تصاویر بیشتر
2,https://upload.wikimedia.org/wikipedia/commons...,لوتار فون مایر,یولیوس لوتار مایر
3,https://upload.wikimedia.org/wikipedia/commons...,فینال مسابقات باشگاهی قهرمانی آسیا ۱۹۷۰,تماشاگران از ساعت‌ها پیش از آغاز بازی در ورزشگ...
4,https://upload.wikimedia.org/wikipedia/commons...,شیء ناشناس پرنده,عکس گرفته‌شده از تصویر احتمالی یک بشقاب‌پرنده ...
...,...,...,...
137165,https://upload.wikimedia.org/wikipedia/commons...,کوزره,باغ انگور در روستای کوزره
137166,http://upload.wikimedia.org/wikipedia/commons/...,خیابان بریک‌لین,برخی از رستوران‌های خیابان بریک‌لین .
137167,https://upload.wikimedia.org/wikipedia/commons...,لیناردو نایتو دی ماکیدو,لیناردو نایتو دی ماکیدو
137168,https://upload.wikimedia.org/wikipedia/commons...,صالح کوتاه,نقشه دیتر امان فرانسوی از اندیکا در سال ۱۹۱۰ (...


In [ ]:
def convertToJpg(parent_dir, fname):
    """
    Convert png, webp, jpeg to .jpg
    """
    im = Image.open(parent_dir + fname)
    rgb_im = im.convert('RGB')
    postfix = fname.split('.')[-1]
    c_fname = fname.replace(postfix, 'jpg') # filename with .jpg postfix
    rgb_im.save(parent_dir + c_fname)
    os.remove(parent_dir + fname) # delete original image.
    print(f'{c_fname} ............converted to jpg.')
    return c_fname

## Download images

In [ ]:
def downloadImage(url, parent_dir, filename):
  """
  Given a url and filename, it'll download image and save it into path/{passed filename}.
  """

  opener = urllib.request.build_opener()
  opener.addheaders = [('User-Agent','Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1941.0 Safari/537.36')]
  urllib.request.install_opener(opener)

  try:
    postfix = url.split('.')[-1].lower()
    filename = str(filename) + '.' + postfix
    urllib.request.urlretrieve(url, parent_dir + filename)
    print(f"{filename} .............Downloaded!")
    status = True
    return status, filename
  except Exception as e:
    status = False
    print(e)
    return status, np.nan

In [ ]:
def download(urls, path):
  for idx in range(len(urls)):
    # Dowload url and save status:(True / False)
      isDownloaded, dn_fname = downloadImage(urls[idx], parent_dir=path, filename=str(idx))
      # check if file downloaded correctly and its format isn't jpg:
      if isDownloaded and not dn_fname.endswith('.jpg'):
        filename = convertToJpg(parent_dir=path, fname=dn_fname)
        df.loc[idx, 'filename'] = filename # append filename to dataset
      elif isDownloaded and dn_fname.endswith('.jpg'):
        df.loc[idx, 'filename'] = dn_fname # append filename to dataset
      else:
        df.loc[idx, 'filename'] = dn_fname # append filename to dataset

# List of urls retrieved from dataset.
urls = list(df['image_url'])
print("Number of urls : {}".format(len(urls)))

# Directory that images downloaded in.
download_here = './wikipedia_images/'

download(urls, download_here)

Number of urls : 137170


In [ ]:
df.to_csv("./wikipedia_facap.csv", index=False)